Importing libraries for standard work

In [1]:
import hashlib                   #To deal with hashes
import datetime                  #To add time to the transactions
import collections               #To store data of block, txn, etc in dictionary form (dictionary being one of its methods)
import binascii                  #converting numbers from binary to ascii and reverse
import string                    #string manipulation
import os                        #To use one of its functions to join strings
import time                      #Converting time to unix timestamp for 'Nonce'


In [2]:
#Just a suite of ciphers that we will be using here.

import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.Signature import pkcs1_15
from Crypto.PublicKey import RSA

from Crypto.Cipher import PKCS1_v1_5 as cipher_PK
#Since the Signature and Cipher have the same function name


**Defining a Client**

A client could be a sender or a receiver of txns

A client is identified with their public key

Private Key is used to sign the message

In [3]:
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        self.privateKey = RSA.generate(1024, random)
        #Generates a random key everytime you call it
        
        self.publicKey = self.privateKey.publickey()
        #publickey() function creates a public key corresponding to given object (in this case 'self.privateKey')
        
        self.signer = pkcs1_15.new(self.privateKey)
        #This function is used to sign the message using RSA key
        #Note that I am keeping this here right now only for testing purposes so that I will be able to test each txn or function individually
        #We will be bringing this forward again into the Transaction class as a function!
        
        return None
        #Even though this statement is not necessary (since python compiler automatically considers it as none)
        #We added this for completeness (and also the auto indentation)

    @property
    def identity(self):
        return (binascii.hexlify(self.publicKey.exportKey(format="DER")).decode('ascii'))
        #Here we are returning the public key
        #We first get the public key in text format, export it according to Distinguished Encoding Rules,
        #Then convert the byte format to hexa-decimal format and then finally to ascii format to get the following output
        
    #Note that the starting and ending few characters will be same, while others are different
    #Because of nature of RSA format, the key will have some common text/characters whenever
    #So, we may have to find a way to deal with that, till then, this seems good!

In [4]:
Ishan = Client()
Ishan.identity

'30819f300d06092a864886f70d010101050003818d0030818902818100bc721b790d9177c5277d45f5eef54b9278f415a872bbc05f68da95364a62f7d37e2a07ced50df8b14951c2c0de1bac1a08480829eb10949b3b5ae113a354c351a13d0c220af089f10514abb542a04cc741cfe0a61f1177d59a398589c16f7ee28911988f2408600f34f42e04b5f6290e4ad71c043e0735ef2c5abb5397b4511d0203010001'

In [5]:
Ishan.privateKey

RsaKey(n=132331093037467302063815404335752078874159021828212421772901105749754555824569534770790990236207577158559038402144663250769251862122808328551935705062020973209300167588014007895690252128635827129404640801440718535646195495608245387368810335088746171834971476253747081175025017929079217969944470874501879714077, e=65537, d=2850871626411996955648124873382802055470626010026980682250078953265383808543588747988271072328370928158304119228698241463549277068322559632232035438393397711119428634187003336856026571172672848162869330183247112447777625054737065210072599143961428931827684002448776698249909222564745527851492972011273642185, p=11363173733648722297167635895898858772081416394970014209460204655334470340874163953996219754906140312076611843376096776171925864003901139563802156093743379, q=11645610296849320819948097044125701689784775341240650029012766894591502708889154381774198216525137042624763009196355707963513642082572597399310198839982863, u=5709073775435149870901301835342180476127

**Defining Transaction**

Now, for our current project we will only be considering the transaction
Where we send some money from 1 person to another

So, we kow that there must be 1 sender and 1 receiver and the value that is being sent. These would be the values that are sent to the Transaction Class.
However, there would be some implicit values like the timestamp of the transaction. Even if you initialize the transaction many times, the signature will be different. Cuz timestamp is different!

In [6]:
class Transaction:
    def __init__(self, sender, receiver, value):
        self.sender = sender
        self.receiver = receiver
        self.value = value
        self.time = datetime.datetime.now()
        self._signature = ''
        Transaction.signTransaction(self)
        #Removing the need to call the function again later :)
        
        return None
    
    
    def signTransaction(self):
        #Here we are going to sign the transaction
        #What happens here is that we take signature of sender by using PKCS1 v1.5 
        #We then hash the transaction using SHA
        #Finally, we sign the transaction using signature created in step1
        
        self.h = SHA.new(str(self.toDict()).encode('utf8')) #Step 1
        #First, we take a hash of the entire message
        #basic str converts to string while utf8 converts it into bytes format.
        #This hash will be used to sign as well as later verify the signature
        

        #Writing the code snippet for Genesis Block
        if self.sender == "Genesis":
            self._signature = pkcs1_15.new(Ishan.privateKey).sign(self.h)
            #In this case, the receiver, which would be assumed to be the Genesis block creator will be signing the transaction.
        #-----------------------------------------------------------------------------------------------------------------------------------
        else:
            self._signature = pkcs1_15.new(self.sender.privateKey).sign(self.h)
            #signing the transaction hash with sender's private key to generate signature

        return None
    

    #This is a function we have created to display the Transaction information
    def toDict(self):
        if self.sender == "Genesis":
            #While creating transactions, we will set sender as Genesis, so that program will know it is the Genesis transaction and append it accordingly.
            identity = "Genesis"
        else:
            identity = self.sender.identity
            
        return collections.OrderedDict ({
            #collections.OrderedDict is a function in python that converts what you enter, into a dictionary based structure
            #Make sure to store the return value of this function into a dict based variable
            #We will be using this to display the details of the transaction
            'Sender' : identity,
            'Recipient' : self.receiver,
            'Amount' : self.value,
            'Timing' : self.time,
        })
    
    def showTxnDetails(self):    
        txnInfo = self.toDict()
        #We chose to keep the toDict function as it will be used for signing purposes
        #For displaying purposes, showTxnDetails will be used.
        _strAppend = []
        # A list of dynamic type (we will be storing strings) is created to append the transaction information along with some other sentences
        # We will be using this along with 'os' library functions to display all of info in one string variable
        
        _strAppend.append("Sender = " + txnInfo['Sender'])
        _strAppend.append('------------------------')
        _strAppend.append("Receiver = " + txnInfo['Recipient'])
        _strAppend.append('-------------------------')
        _strAppend.append("Amount = " + str(txnInfo['Amount']))
        _strAppend.append('-----------------------------')
        _strAppend.append("TimeStamp = " + str(txnInfo['Timing']))
        _strAppend.append('=================================================================')
        
        return os.linesep.join(_strAppend)
        #Finally, we join the list of strings into 1 string.
        #This function's return value is to be stored into another variable which will then be used as a string
    
    
    @property
    #Returns signature in ascii string format.
    def signature(self):
        return binascii.hexlify(self._signature).decode('ascii')


In [7]:
#Defining another client to be the recipient for now
Varun = Client()

Note that in a blockchain environment (or at least certainly in Bitcoin Blockchain)..

When you send a BTC to another person you need following things:

The public address of the receiver

Amount with self (We will validate that in another place)

Sign the transaction with your own private key


So, when creating a transaction, 
Remember to pass the reciever as receiver.identity and not just receiver

Note that the identity property of the client returns their Public Key, hence so.

(For the case of this project, we haven't added any further protection to the public key like in Bitcoin blockchain. So, we are directly passing the public key forward. We could choose to take RIPEMD160 later)

In [8]:
hihi = Transaction(Ishan, Varun.identity, 12)

Now, to **display the transaction**, we could directly store the return value of showTxnDetails inside a new variable and print it



x= hihi.showTxnDetails()
print(x)

**This is a demonstrative code**

*Save it as 'Code' in order to run this, otherwise it will be in 'Markdown' format*

Now, we are set to install a queue which will hold the transactions


In [9]:
transactions = []

Let us also create some more clients

In [10]:
Anagha = Client ()
Makarand = Client() 
#Ishan and Varun already exist

Now, we create multiple transactions and append them to our transactions list

**CREATING OBJECT OF CLASS TRANSACTION!**

To see all of our transactions, we could run the following code

for transaction in transactions:
    txnMark = transaction.showTxnDetails()
    #Just a name to differentiate variables
    print(txnMark)
    
**RUN THIS CODE IF YOU WANT TO CHECK IF TRANSACTIONS have been appended carefully and correctly**

*It is currently in markdown and you will have to change it to 'Code' in order to make it run*
Also, remove these comments.

In [11]:

lastTransactionIndex = 0
#At the start, this value will be 0
#Later on, we go on incrementing this value as we iterate through the transactions

**NOW WE HAVE TO ADD GENESIS BLOCK**

**AND THEN add its hash to the new block as previous Block Hash**

In [12]:
class Block:
    def __init__ (self):
        self.includedTxns = []
        now = datetime.datetime.now()
        self.nonce = round(time.mktime(now.timetuple()))      #Setting nonce to be current unix time of when object of this class will be created.
                                                              #Added the round function so as to remove the last decimal point
        self.blockHash = ""
        self.prevBlockHash = ""
        self.merkleRoot = ""
        
    def _searchTxns(self):
        count = 0
        global lastTransactionIndex
        for txn in range(lastTransactionIndex, len(transactions)):
            
            #Step 1: We take the txn from list
            tempTxn = transactions[lastTransactionIndex]
            
            h = SHA.new(str(tempTxn.toDict()).encode('utf8'))
            #Step 2: Calculating hash of Txn again for later verification
            
            #Step 3: We verify the transactions within a try-catch block
            try:
                if tempTxn.sender == "Genesis":
                    pkcs1_15.new(Ishan.publicKey).verify(h, tempTxn._signature)
                
                #Configured the signature verification to verify Genesis transaction from my name 
                #--------------------------------------------------------------------------------------------------------------------------
            
                else:
                    pkcs1_15.new(tempTxn.sender.publicKey).verify(h, tempTxn._signature)
                    #Lots of things were gone through
                    #In the last case, we had to use the bytes form of signature as well (and not just hash's)
                    #Here, we have taken the Public Key of sender, the hash of transaction and its bytes format signature!
                
                print("Signature is Valid!!!")
                
            except (ValueError, TypeError):
                print("The signature was not valid :P")
                lastTransactionIndex += 1
                #Since it will not perform further action
                continue
                
            self.includedTxns.append(tempTxn)
            lastTransactionIndex += 1
            count += 1
            if(count == 3):
                break
            #Making sure that txn count in Block does not exceed 3
        return None
    
    
    def _startMining(self):
        self.blockHash = mine(self, 2)
        str1 = self._includedTxn
        self.merkleRoot = sha256(str1)             #Makeshift merkle root, hash of all transactions of the block in contiguous form
        return None
    
    
    @property
    def _includedTxn(self):
        txnc = 1
        txInfo = []
        for _in_txn in self.includedTxns:
            txInfo.append("Transaction "+ str(txnc))
            txInfo.append(_in_txn.showTxnDetails())
            txnc += 1
        return os.linesep.join(txInfo)
    #This property now returns a string 
    #It has to be stored into a variable and then printed (If want to display correctly)
    
    @property
    def blockHeader(self):
        blockDict = collections.OrderedDict({ 
            'merkleRoot' : self.merkleRoot,
            'Nonce' : str(self.nonce),
            'pBlockHash' : self.prevBlockHash
        })
        
        print("Merkle Root = " + blockDict['merkleRoot'])
        print("Nonce = " + blockDict['Nonce']) 
        print("Previous Block Hash = " + blockDict['pBlockHash'])
        return None

        
#Defining a variable to store previous Block's header into new block
lastBlockHash = ""

**NEXT STEP**


MINING Function!

**Note**
Now, if we put the mining function within the block itself.
 It is possible that the miners may set multiple blocks to mine at the same time. Hoping to reach the nonce quicker
 We do not want that. So, we create a separate function outside the block class which will be called
 And then, we can apply a locking function to the miner or the mining function!
 
 As an additional thing, I believe we can go for an attribute to the Client which would determine if it is a miner or not
 Or create a separate 'Miner' class that is invoked from the Client class once a specific condition has been satisfied.
 The miner class will then have the locking functionality. As they lock onto 1 block which they have selected for mining.

For our mining function, we will be using the sha256 hash
However, by default it provides in a way not usable. So we have to act accordingly

In [13]:
def sha256(msg):
    return hashlib.sha256(msg.encode('ascii')).hexdigest()

In [14]:
def mine(message, diff = 1):
    assert diff >= 1
    prefix = '0' * diff
    #It seems this sort of thing can be done in python (making it easier huh)
  
    
        
    bHash = ''
    iteration = 0
    while (not bHash.startswith(prefix)):
        # 'not' is a keyword in Python used for negation. It will reverse the boolean value that is returned.
        #That is, for this while loop, we want it to keep going as long as we do not have a Nonce that starts with prefix
        # i.e it will stop once we find the Nonce with correct prefix
        
        bHash = sha256(str(hash(message)) +str(iteration))
        iteration += 1

    print("After " +str(iteration)+" iterations, Block Hash found is : " +bHash)
    return str(bHash)


**NEXT TASK IS TO CREATE BLOCKCHAIN!**

In [15]:
PyCoin = []
#Name of our blockchain!

In [16]:
print("Do you want it's transactions to be shown as well with the blockchain info?")


Do you want it's transactions to be shown as well with the blockchain info?


In [17]:
response = await input("Enter 'Y' if yes, anything else if no.\n")
# Key was to use 'await'!!
# Await keyword changes it from pyodide... something to just the input that we provide!
#This input can then be typecasted into integer and the like as well!
#WohooO!!!!

Enter 'Y' if yes, anything else if no.
 Y


**Note that if you speed run this, by holding shift and enter**

**It is likely that you will send the next cells to execute too, possibly making them run twice**

**Resulting in 2 Genesis blocks (with possibly different transactions in them). It's funny, but yeah.**

**Restart the Kernel to rectify this. (It's that refresh button to the left of 'Code' or 'Markdown' or 'Raw' whatever you see. Should be right below the tab of this notebook.**

In [18]:
def printBlockchain(self):
    print("Number of blocks in the chain: " +str(len(self)))
    global response
    for x in range(len(self)):
        block_temp = self[x]
        print( "Block # " + str(x))
        print(block_temp.blockHeader)
        
        if (response == 'Y'):
            print("Since the choice was to show transactions inside block as well")
            print("")

            for txn in range(len(block_temp.includedTxns)):
                temp_txn = block_temp.includedTxns[txn]
                print(temp_txn.showTxnDetails())
                print('___________________________________________________________________________________')
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        else:
            print("All right! You chose not to show transactions!")

In [19]:
printBlockchain(PyCoin)

Number of blocks in the chain: 0


**Creating the Genesis Block**

**First, we need the Genesis Transaction**

In [20]:
t_G = Transaction("Genesis", Ishan.identity, 500.0)

In [21]:
block0 = Block()

In [22]:
block0.prevBlockHash = "Genesis."

In [23]:
t_G.showTxnDetails()

'Sender = Genesis\n------------------------\nReceiver = 30819f300d06092a864886f70d010101050003818d0030818902818100bc721b790d9177c5277d45f5eef54b9278f415a872bbc05f68da95364a62f7d37e2a07ced50df8b14951c2c0de1bac1a08480829eb10949b3b5ae113a354c351a13d0c220af089f10514abb542a04cc741cfe0a61f1177d59a398589c16f7ee28911988f2408600f34f42e04b5f6290e4ad71c043e0735ef2c5abb5397b4511d0203010001\n-------------------------\nAmount = 500.0\n-----------------------------\nTimeStamp = 2023-06-29 19:15:30.711000\n================================================================='

**Now, the task is to add this transaction to a block and hail it as the genesis block!**

In [24]:
transactions.append(t_G)

In [25]:
block0._searchTxns()
block0._startMining()

PyCoin.append(block0)
#I don't need a separate functionality to make sure that the earlier functions are successful because if they are not then the next line won't be executed.
#Also, once the block is mined, it is to be added to the blockchain immediately in this project.

Signature is Valid!!!
After 320 iterations, Block Hash found is : 00638c87e5d8ac921de236fa62d87fd1c3f10ec1e07891267c1911845e5d2a37


**Now, let's have a look at our blockchain!**

In [26]:
printBlockchain(PyCoin)

Number of blocks in the chain: 1
Block # 0
Merkle Root = ea746bb8affe4d95c8000c81bdb56c2b23ae30587a73c240220bb1898e228de4
Nonce = 1688046331
Previous Block Hash = Genesis.
None
Since the choice was to show transactions inside block as well

Sender = Genesis
------------------------
Receiver = 30819f300d06092a864886f70d010101050003818d0030818902818100bc721b790d9177c5277d45f5eef54b9278f415a872bbc05f68da95364a62f7d37e2a07ced50df8b14951c2c0de1bac1a08480829eb10949b3b5ae113a354c351a13d0c220af089f10514abb542a04cc741cfe0a61f1177d59a398589c16f7ee28911988f2408600f34f42e04b5f6290e4ad71c043e0735ef2c5abb5397b4511d0203010001
-------------------------
Amount = 500.0
-----------------------------
TimeStamp = 2023-06-29 19:15:30.711000
___________________________________________________________________________________
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [27]:
t1 = Transaction(Ishan, Anagha.identity, 5.0)
t2 = Transaction(Makarand, Varun.identity, 7.0)
t3 = Transaction(Varun, Ishan.identity, 3.0)
t4 = Transaction(Anagha, Makarand.identity, '15.0')
t5 = Transaction(Anagha, Ishan.identity, '16.0')
t6 = Transaction(Varun, Makarand.identity, '8.0')
t7 = Transaction(Makarand, Anagha.identity, '12.0')


transactions.append(t1)
transactions.append(t2)
transactions.append(t3)
transactions.append(t4)
transactions.append(t5)
transactions.append(t6)
transactions.append(t7)

**Beyond this, we can now add newer blocks.**

So, we will be asking the user if they wish to create a new block.

In [28]:
#This function is useful so that now we don't have to look at the latest block 
#and reference its block hash to add into current block as the Previous Block Hash!

def setPrevBlockHash(self):
    t_block = PyCoin[-1]
    self.prevBlockHash = t_block.blockHash

In [29]:
block1 = Block()
setPrevBlockHash(block1)

#This function will have to be called whenever we create a new block.

In [30]:
block1.prevBlockHash

'00638c87e5d8ac921de236fa62d87fd1c3f10ec1e07891267c1911845e5d2a37'

In [31]:
block1._searchTxns()
block1._startMining()

PyCoin.append(block1)
#As we know, the block will get appended as soon as the 1st two conditions are satisfied.

Signature is Valid!!!
Signature is Valid!!!
Signature is Valid!!!
After 106 iterations, Block Hash found is : 0074df41b796f32706e07545e6b78656dfe5e7adf029e6370acf514b0433f059


In [32]:
block2 = Block()
setPrevBlockHash(block2)
block2.prevBlockHash
#Not necessary, but just added this here, to show the previous block hash here as well.

block2._searchTxns()
block2._startMining()

PyCoin.append(block2)

Signature is Valid!!!
Signature is Valid!!!
Signature is Valid!!!
After 240 iterations, Block Hash found is : 00aa319d73d90d5ffb75c67db1e7cb9c9a35d81af30d41913485c14ca95777bb


**We can then call the Print BlockChain function**

**Or we could add newer functions, whichever, it is your choice!**

In [33]:
#Turn this into 'Code' (currently set to 'Raw') in order to make this run.

printBlockchain(PyCoin)

Number of blocks in the chain: 3
Block # 0
Merkle Root = ea746bb8affe4d95c8000c81bdb56c2b23ae30587a73c240220bb1898e228de4
Nonce = 1688046331
Previous Block Hash = Genesis.
None
Since the choice was to show transactions inside block as well

Sender = Genesis
------------------------
Receiver = 30819f300d06092a864886f70d010101050003818d0030818902818100bc721b790d9177c5277d45f5eef54b9278f415a872bbc05f68da95364a62f7d37e2a07ced50df8b14951c2c0de1bac1a08480829eb10949b3b5ae113a354c351a13d0c220af089f10514abb542a04cc741cfe0a61f1177d59a398589c16f7ee28911988f2408600f34f42e04b5f6290e4ad71c043e0735ef2c5abb5397b4511d0203010001
-------------------------
Amount = 500.0
-----------------------------
TimeStamp = 2023-06-29 19:15:30.711000
___________________________________________________________________________________
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Block # 1
Merkle Root = 0979a96c1fffd07fb03be79298ac82e00010f3be5744ab9a7b63c9cc0df0e9b9
Nonce